In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import ShortType
from delta.tables import DeltaTable

CATALOGO_ORIGEM = "v_credit"
SCHEMA_ORIGEM = "bronze"
TABELA_ORIGEM = "base_atendentes"

CATALOGO_DESTINO = "v_credit"
SCHEMA_DESTINO = "silver"
TABELA_DESTINO = "tb_atendente"
TABELA_INVALIDOS_DESTINO = "tb_atendente_invalidos"

nome_tabela_origem = f"{CATALOGO_ORIGEM}.{SCHEMA_ORIGEM}.{TABELA_ORIGEM}"
nome_tabela_destino = f"{CATALOGO_DESTINO}.{SCHEMA_DESTINO}.{TABELA_DESTINO}"

### Leitura Incremental do Bronze
Le apenas a ultima carga (max ingestion_timestamp), aplica limpeza:
- Renomeia colunas para padrao Silver (cd_, nm_, nu_)
- Trim em strings
- Cast de tipos
- Deduplicacao por PK (cd_atendente)

### Validacao de Qualidade
Adiciona flags de validacao:
- **flag_id_valido**: cd_atendente IS NOT NULL
- **flag_nome_valido**: nm_atendente IS NOT NULL AND length > 1
- **flag_nivel_valido**: nu_nivel IN (1, 2)
- **flag_qualidade**: "OK" se todas flags TRUE, senao "ERRO"

Split em df_validos e df_invalidos.

### MERGE de Registros Validos
Faz MERGE (UPSERT) em tb_atendente:
- **MATCHED**: Atualiza registro existente
- **NOT MATCHED**: Insere novo registro

Operacao idempotente (pode ser re-executada sem gerar duplicatas).

### OVERWRITE de Registros Invalidos
Sobrescreve tb_atendente_invalidos com registros rejeitados da ultima carga.
Contem flags de validacao para rastreabilidade de problemas.

### Verificacao: Chamados com Atendente
Conta quantos chamados foram atendidos por humanos (cd_atendente NOT NULL).

### Verificacao: Total de Chamados
Conta o total de chamados (inclui automatizados onde cd_atendente = NULL).

### Configuracao
Define origem (Bronze), destino (Silver) e tabela de auditoria (invalidos).

In [0]:
df_bronze = spark.read.table(nome_tabela_origem)

max_dt_ingestao = (
    df_bronze
    .agg(F.max(F.col("ingestion_timestamp")).alias("max_ts"))
    .first()["max_ts"]
)

df_limpo = (
    df_bronze
    .filter(F.col("ingestion_timestamp") == F.lit(max_dt_ingestao))
    .select(
        F.col("id_atendente").alias("cd_atendente"),
        F.trim(F.col("nome_atendente")).alias("nm_atendente"),
        F.col("nivel_atendimento").cast(ShortType()).alias("nu_nivel"),
        F.col("ingestion_timestamp").alias("dt_ingestion"),
        F.col("origem").alias("dc_origem")
    )
    .dropDuplicates(["cd_atendente"])
)

In [0]:
df_validacao = (
    df_limpo
    .withColumn("flag_id_valido", F.col("cd_atendente").isNotNull())
    .withColumn("flag_nome_valido", 
        F.col("nm_atendente").isNotNull() & (F.length(F.col("nm_atendente")) > 1)
    )
    .withColumn("flag_nivel_valido", 
        F.col("nu_nivel").isNotNull() & 
        (F.col("nu_nivel") >= 1) & 
        (F.col("nu_nivel") <= 2)
    )
    .withColumn("flag_qualidade",
        F.when(
            F.col("flag_id_valido") &
            F.col("flag_nome_valido") &
            F.col("flag_nivel_valido"),
            F.lit("OK")
        ).otherwise(F.lit("ERRO"))
    )
)

df_validos = df_validacao.filter(F.col("flag_qualidade") == "OK")
df_invalidos = df_validacao.filter(F.col("flag_qualidade") == "ERRO")

df_silver = df_validos.select(
    "cd_atendente",
    "nm_atendente", 
    "nu_nivel",
    "dt_ingestion",
    "dc_origem"
)


In [0]:
delta_table = DeltaTable.forName(spark, nome_tabela_destino)

delta_table.alias("destino").merge(
    df_silver.alias("origem"),
    "destino.cd_atendente = origem.cd_atendente"
).whenMatchedUpdateAll(
).whenNotMatchedInsertAll(
).execute()

print(f"✅ Tabela {nome_tabela_destino} atualizada com sucesso !")

In [0]:
nome_tabela_invalidos = f"{CATALOGO_DESTINO}.{SCHEMA_DESTINO}.{TABELA_INVALIDOS_DESTINO}"

df_invalidos.write.format("delta").mode("overwrite").saveAsTable(nome_tabela_invalidos)

print(f" ✅ Tabela {nome_tabela_invalidos} atualizada para auditoria")

In [0]:
query = """
SELECT COUNT(*) AS qtd_chamados
FROM v_credit.silver.tb_chamado
WHERE cd_atendente IS NOT NULL
"""

df_result = spark.sql(query)
display(df_result)

In [0]:
query = """
SELECT COUNT(*) AS qtd_chamados
FROM v_credit.silver.tb_chamado
"""

df_result = spark.sql(query)
display(df_result)